In [2]:
import pandas as pd
import requests
from datetime import date, timedelta

# Get Hub Ratings

The betfair ratings model is available on the betfair Hub and if you do a little digging it's served to the site by a (though not officially supported) convenient API.

In [3]:
url = 'https://betfair-data-supplier-prod.herokuapp.com/api/widgets/kash-ratings-model/datasets?date=2021-03-21&presenter=RatingsPresenter&json=true'


rawResponse = requests.get(url)

In [4]:
json = rawResponse.json()
json

{'meetings': [{'name': 'BAIRNSDALE',
   'bfExchangeEventId': '30365338',
   'races': [{'bfExchangeMarketId': '1.180806537',
     'name': 'R1 1000m Mdn',
     'number': 1,
     'runners': [{'bfExchangeSelectionId': 25088155,
       'name': '2. Fortyfive Parklane',
       'ratedPrice': 5.16,
       'number': 0},
      {'bfExchangeSelectionId': 38860000,
       'name': '3. Im The Boldest',
       'ratedPrice': 17.88,
       'number': 0},
      {'bfExchangeSelectionId': 38860001,
       'name': '4. The Javelin',
       'ratedPrice': 4.57,
       'number': 0},
      {'bfExchangeSelectionId': 36136786,
       'name': '5. Delavigne',
       'ratedPrice': 22.72,
       'number': 0},
      {'bfExchangeSelectionId': 38860002,
       'name': '6. Epona Pink',
       'ratedPrice': 10.68,
       'number': 0},
      {'bfExchangeSelectionId': 17550584,
       'name': '7. Equishim',
       'ratedPrice': 138.17,
       'number': 0},
      {'bfExchangeSelectionId': 25898835,
       'name': '8. The Trance

In [5]:
hubList = []
for meeting in json['meetings']:
    for race in meeting['races']:
        for runner in race['runners']:
            hubList.append(
                {
                    'track': meeting['name'],
                    'race_number': race['number'],
                    'race_name': race['name'],
                    'market_id': race['bfExchangeMarketId'],
                    'selection_id':  runner['bfExchangeSelectionId'],
                    'selection_name': runner['name'],
                    'model_odds': runner['ratedPrice']
                }
            )


In [6]:
hubDf = pd.DataFrame(hubList)

hubDf

,track,race_number,race_name,market_id,selection_id,selection_name,model_odds
0,BAIRNSDALE,1,R1 1000m Mdn,1.180806537,25088155,2. Fortyfive Parklane,5.16
1,BAIRNSDALE,1,R1 1000m Mdn,1.180806537,38860000,3. Im The Boldest,17.88
2,BAIRNSDALE,1,R1 1000m Mdn,1.180806537,38860001,4. The Javelin,4.57
3,BAIRNSDALE,1,R1 1000m Mdn,1.180806537,36136786,5. Delavigne,22.72
4,BAIRNSDALE,1,R1 1000m Mdn,1.180806537,38860002,6. Epona Pink,10.68
...,...,...,...,...,...,...,...
513,TE AROHA,7,R7 1400m Mdn,1.180806972,38848791,12. Polly Power,26.50
514,TE AROHA,7,R7 1400m Mdn,1.180806972,38848792,13. Extersea,12.35
515,TE AROHA,7,R7 1400m Mdn,1.180806972,6110818,1. Gold Winner,2.26
516,TE AROHA,7,R7 1400m Mdn,1.180806972,38848793,14. Red Rushin,94.28


# Running For A Set Of Dates

Now we just need to run it over multiple dates. Let's say Jan 2021 onwards

In [7]:
def getHubRatings(dte):
    
    url = 'https://betfair-data-supplier-prod.herokuapp.com/api/widgets/kash-ratings-model/datasets?date={}presenter=RatingsPresenter&json=true'.format(dte)
    
    responseJson = requests.get(url).json()
        
    hubList = []
    
    if not responseJson:
        return(None)
    
    for meeting in responseJson['meetings']:
        for race in meeting['races']:
            for runner in race['runners']:
                hubList.append(
                    {
                        'date': dte,
                        'track': meeting['name'],
                        'race_number': race['number'],
                        'race_name': race['name'],
                        'market_id': race['bfExchangeMarketId'],
                        'selection_id':  runner['bfExchangeSelectionId'],
                        'selection_name': runner['name'],
                        'model_odds': runner['ratedPrice']
                    }
                )
                
    return(pd.DataFrame(hubList))

In [8]:
getHubRatings(date(2021,3,5))

,date,track,race_number,race_name,market_id,selection_id,selection_name,model_odds
0,2021-03-05,CRANBOURNE,1,R1 1000m 3yo,1.180059178,38687451,1. Faithless,15.61
1,2021-03-05,CRANBOURNE,1,R1 1000m 3yo,1.180059178,38687452,2. Joyous Tea,27.52
2,2021-03-05,CRANBOURNE,1,R1 1000m 3yo,1.180059178,38687453,3. Madame Bolli,3.43
3,2021-03-05,CRANBOURNE,1,R1 1000m 3yo,1.180059178,38687454,4. Merlins Charm,3.24
4,2021-03-05,CRANBOURNE,1,R1 1000m 3yo,1.180059178,38687455,5. Okay To Pay,5.74
...,...,...,...,...,...,...,...,...
648,2021-03-05,STAWELL,8,R8 2000m Hcap,1.180058800,22019438,9. So Belafonte,50.03
649,2021-03-05,STAWELL,8,R8 2000m Hcap,1.180058800,8236063,10. Fratello,24.33
650,2021-03-05,STAWELL,8,R8 2000m Hcap,1.180058800,38687695,11. Elvison,30.03
651,2021-03-05,STAWELL,8,R8 2000m Hcap,1.180058800,26909399,12. Its Choux Time,53.01


In [9]:
dateDFList = []
dateList = pd.date_range(date(2021,1,1),date.today()-timedelta(days=1),freq='d')

for dte in dateList:
    print(dte)
    dateDFList.append(getHubRatings(dte))

2021-01-01 00:00:00
2021-01-02 00:00:00
2021-01-03 00:00:00
2021-01-04 00:00:00
2021-01-05 00:00:00
2021-01-06 00:00:00
2021-01-07 00:00:00
2021-01-08 00:00:00
2021-01-09 00:00:00
2021-01-10 00:00:00
2021-01-11 00:00:00
2021-01-12 00:00:00
2021-01-13 00:00:00
2021-01-14 00:00:00
2021-01-15 00:00:00
2021-01-16 00:00:00
2021-01-17 00:00:00
2021-01-18 00:00:00
2021-01-19 00:00:00
2021-01-20 00:00:00
2021-01-21 00:00:00
2021-01-22 00:00:00
2021-01-23 00:00:00
2021-01-24 00:00:00
2021-01-25 00:00:00
2021-01-26 00:00:00
2021-01-27 00:00:00
2021-01-28 00:00:00
2021-01-29 00:00:00
2021-01-30 00:00:00
2021-01-31 00:00:00
2021-02-01 00:00:00
2021-02-02 00:00:00
2021-02-03 00:00:00
2021-02-04 00:00:00
2021-02-05 00:00:00
2021-02-06 00:00:00
2021-02-07 00:00:00
2021-02-08 00:00:00
2021-02-09 00:00:00
2021-02-10 00:00:00
2021-02-11 00:00:00
2021-02-12 00:00:00
2021-02-13 00:00:00
2021-02-14 00:00:00
2021-02-15 00:00:00
2021-02-16 00:00:00
2021-02-17 00:00:00
2021-02-18 00:00:00
2021-02-19 00:00:00


In [11]:
hubRatings = pd.concat(dateDFList)


In [12]:
hubRatings

,date,track,race_number,race_name,market_id,selection_id,selection_name,model_odds
0,2021-02-18,DOOMBEN,1,R1 1200m 3yo,1.179418181,38523320,11. Vast Kama,34.28
1,2021-02-18,DOOMBEN,1,R1 1200m 3yo,1.179418181,38523319,10. Triptonic,21.22
2,2021-02-18,DOOMBEN,1,R1 1200m 3yo,1.179418181,35773035,9. Right Reason,10.23
3,2021-02-18,DOOMBEN,1,R1 1200m 3yo,1.179418181,38523318,8. Off Road,40.75
4,2021-02-18,DOOMBEN,1,R1 1200m 3yo,1.179418181,38523317,7. More Than Value,77.49
...,...,...,...,...,...,...,...,...
660,2021-03-28,WELLINGTON,8,R8 1100m Hcap,1.181113618,352158,10. Worldly Pleasure,43.14
661,2021-03-28,WELLINGTON,8,R8 1100m Hcap,1.181113618,38936999,12. Major Time,8.50
662,2021-03-28,WELLINGTON,8,R8 1100m Hcap,1.181113618,38937000,13. Golden Gorge,13.13
663,2021-03-28,WELLINGTON,8,R8 1100m Hcap,1.181113618,38937001,14. Sons Of Bourke,21.49


In [13]:
hubRatings['date'].min()

Timestamp('2021-02-18 00:00:00')